## CSc 8830: Computer Vision - Assignment 4

1. Implement an application (must run on web or as an app on mobile device) using the stereo camera where it will recognize, track and estimate dimensions (at least 2D) of any object within 3m distance and inside field-of-view to the camera. You can use barcodes or text recognition tools for identification. However, the entire object must be tracked (not just the barcode or text). Machine/Deep learning tools are NOT allowed.

In [5]:
import cv2
import math
import imutils
import numpy as np
import depthai as dai
from imutils import perspective
from imutils import contours
from scipy.spatial import distance as dist

def detect_qr(frame):
    qcd = cv2.QRCodeDetector()
    ret_qr, decoded_info, points, _ = qcd.detectAndDecodeMulti(frame)
    if ret_qr:
        for s, p in zip(decoded_info, points):
            if s:
                frame = cv2.polylines(frame, [p.astype(int)], True, (255, 255, 255), 8)
                cv2.putText(frame, s, (int(p[0][0]), int(p[0][1])-10), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
            else:
                color = (0, 0, 0)
                frame = cv2.polylines(frame, [p.astype(int)], True, color, 8)
    return frame

class ObjTracker:
    def __init__(self):
        self.obj_centers = {}
        self.obj_id_count = 0

    def update(self, frame):
        roi = frame
        obj_detector = cv2.createBackgroundSubtractorMOG2(history=100, varThreshold=40)
        mask = obj_detector.apply(roi)
        _, mask = cv2.threshold(mask, 254, 255, cv2.THRESH_BINARY)
        contours, _ = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        detections = []
        for cnt in contours:
            area = cv2.contourArea(cnt)
            if area > 100:
                x, y, w, h = cv2.boundingRect(cnt)
                detections.append([x, y, w, h])

        tracked_boxes_ids = self._track_objs(detections)
        for box_id in tracked_boxes_ids:
            x, y, w, h, obj_id = box_id
            cv2.rectangle(roi, (x, y), (x + w, y + h), (0, 255, 0), 3)

        return roi

    def _track_objs(self, obj_rectangles):
        tracked_objects = []
        for rect in obj_rectangles:
            x, y, w, h = rect
            cx = (x + x + w) // 2
            cy = (y + y + h) // 2
            obj_detected = False
            for obj_id, center in self.obj_centers.items():
                dist = math.hypot(cx - center[0], cy - center[1])
                if dist < 25:
                    self.obj_centers[obj_id] = (cx, cy)
                    tracked_objects.append([x, y, w, h, obj_id])
                    obj_detected = True
                    break
            if not obj_detected:
                self.obj_centers[self.obj_id_count] = (cx, cy)
                tracked_objects.append([x, y, w, h, self.obj_id_count])
                self.obj_id_count += 1

        new_obj_centers = {}
        for obj_bb_id in tracked_objects:
            _, _, _, _, obj_id = obj_bb_id
            center = self.obj_centers[obj_id]
            new_obj_centers[obj_id] = center

        self.obj_centers = new_obj_centers.copy()
        return tracked_objects

def midpoint(ptA, ptB):
    return ((ptA[0] + ptB[0]) * 0.5, (ptA[1] + ptB[1]) * 0.5)

class ObjDimensionMarker:
    def __init__(self, obj_width_inches):
        self.obj_width_inches = obj_width_inches
        self.pixelsPerMetric = None

    def mark_obj_dims(self, gray_frame):
        gray = cv2.GaussianBlur(gray_frame, (7, 7), 0)
        edged = cv2.Canny(gray, 50, 100)
        edged = cv2.dilate(edged, None, iterations=1)
        edged = cv2.erode(edged, None, iterations=1)
        cnts = cv2.findContours(edged.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        cnts = imutils.grab_contours(cnts)
        cnts = contours.sort_contours(cnts)[0]
        result_frame = cv2.cvtColor(gray_frame, cv2.COLOR_GRAY2BGR)

        for c in cnts:
            if cv2.contourArea(c) < 100:
                continue
            box = cv2.minAreaRect(c)
            box = cv2.boxPoints(box) if imutils.is_cv2() else cv2.boxPoints(box)
            box = np.array(box, dtype="int")
            box = perspective.order_points(box)
            cv2.drawContours(result_frame, [box.astype("int")], -1, (0, 0, 0), 2)

            (tl, tr, br, bl) = box
            (tltrX, tltrY) = midpoint(tl, tr)
            (blbrX, blbrY) = midpoint(bl, br)
            (tlblX, tlblY) = midpoint(tl, bl)
            (trbrX, trbrY) = midpoint(tr, br)

            cv2.circle(result_frame, (int(tltrX), int(tltrY)), 5, (0, 0, 0), -1)
            cv2.circle(result_frame, (int(blbrX), int(blbrY)), 5, (0, 0, 0), -1)
            cv2.circle(result_frame, (int(tlblX), int(tlblY)), 5, (0, 0, 0), -1)
            cv2.circle(result_frame, (int(trbrX), int(trbrY)), 5, (0, 0, 0), -1)

            cv2.line(result_frame, (int(tltrX), int(tltrY)), (int(blbrX), int(blbrY)), (255, 255, 255), 2)
            cv2.line(result_frame, (int(tlblX), int(tlblY)), (int(trbrX), int(trbrY)), (255, 255, 255), 2)

            dA = dist.euclidean((tltrX, tltrY), (blbrX, blbrY))
            dB = dist.euclidean((tlblX, tlblY), (trbrX, trbrY))

            if self.pixelsPerMetric is None:
                self.pixelsPerMetric = dB / self.obj_width_inches

            dimA_cm = dA / self.pixelsPerMetric * 2.54
            dimB_cm = dB / self.pixelsPerMetric * 2.54

            cv2.putText(result_frame, "{:.1f}in".format(dimA_cm), (int(tltrX - 15), int(tltrY - 10)), cv2.FONT_HERSHEY_SIMPLEX, 0.65, (255, 255, 255), 2)
            cv2.putText(result_frame, "{:.1f}in".format(dimB_cm), (int(trbrX + 10), int(trbrY)), cv2.FONT_HERSHEY_SIMPLEX, 0.65, (255, 255, 255), 2)

        return result_frame

tracker = ObjTracker()

object_marker = ObjDimensionMarker(obj_width_inches=0.995)

pipeline = dai.Pipeline()

stereo = pipeline.createStereoDepth()
stereo.setConfidenceThreshold(255)

left = pipeline.createMonoCamera()
left.setResolution(dai.MonoCameraProperties.SensorResolution.THE_720_P)

right = pipeline.createMonoCamera()
right.setResolution(dai.MonoCameraProperties.SensorResolution.THE_720_P)

left.out.link(stereo.left)
right.out.link(stereo.right)

xoutDepth = pipeline.createXLinkOut()
xoutDepth.setStreamName("depth")
stereo.depth.link(xoutDepth.input)

xoutLeft = pipeline.createXLinkOut()
xoutLeft.setStreamName("left")
left.out.link(xoutLeft.input)

xoutRight = pipeline.createXLinkOut()
xoutRight.setStreamName("right")
right.out.link(xoutRight.input)

cam = pipeline.createColorCamera()
cam.setResolution(dai.ColorCameraProperties.SensorResolution.THE_1080_P)

xout = pipeline.createXLinkOut()
xout.setStreamName("rgb")
cam.video.link(xout.input)

with dai.Device(pipeline) as device:
    depthQueue = device.getOutputQueue(name="depth", maxSize=4, blocking=False)
    leftQueue = device.getOutputQueue(name="left", maxSize=4, blocking=False)
    rightQueue = device.getOutputQueue(name="right", maxSize=4, blocking=False)
    rgbQueue = device.getOutputQueue(name="rgb", maxSize=4, blocking=False)

    device.startPipeline()

    camera_id = 0
    delay = 1

    while True:
        inDepth = depthQueue.get()
        inLeft = leftQueue.get()
        inRight = rightQueue.get()
        inSrc = rgbQueue.get()

        depthFrame = inDepth.getFrame()
        leftFrame = inLeft.getCvFrame()
        rightFrame = inRight.getCvFrame()
        rgbFrame = inSrc.getCvFrame()

        stereoFrame = cv2.hconcat([leftFrame, rightFrame])

        stereoFrame_with_qr = detect_qr(stereoFrame)

        stereoFrame_with_objects_qr = tracker.update(stereoFrame_with_qr)

        gray_frame = cv2.cvtColor(rgbFrame, cv2.COLOR_BGR2GRAY)

        try:
            marked_frame = object_marker.mark_obj_dims(gray_frame)
        except:
            continue

        cv2.imshow("Stereo Vision (Right & Left)", stereoFrame_with_objects_qr)
        cv2.imshow("RGB Camera", marked_frame)

        if cv2.waitKey(1) == ord('q'):
            break

cv2.destroyAllWindows()


/var/folders/cv/flgh8s7960bc7q380pyn0c6m0000gn/T/ipykernel_79026/191203465.py:138: DeprecationWarning: setConfidenceThreshold() is deprecated, Use 'initialConfig.setConfidenceThreshold()' instead
  stereo.setConfidenceThreshold(255)


[1844301031A5061300] [0.1.1] [1.015] [MonoCamera(1)] [error] OV7251 only supports THE_480_P/THE_400_P resolutions, defaulting to THE_480_P
[1844301031A5061300] [0.1.1] [1.016] [MonoCamera(2)] [error] OV7251 only supports THE_480_P/THE_400_P resolutions, defaulting to THE_480_P


/var/folders/cv/flgh8s7960bc7q380pyn0c6m0000gn/T/ipykernel_79026/191203465.py:174: DeprecationWarning: Device(pipeline) starts the pipeline automatically. Use Device() and startPipeline(pipeline) otherwise
  device.startPipeline()
